# FABRIC API Example: Create a Node with Components


## Configure the Environment

In [1]:
import os
import json
import traceback
from getpass import getpass

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_BASTION_USERNAME']='pruth'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'
print('Please input private key passphrase. Press enter for no passphrase.')
os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

Please input private key passphrase. Press enter for no passphrase.
········


In [2]:
slice_private_key_file=os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']
slice_public_key_file=os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']

slice_public_key = None
with open (slice_public_key_file, "r") as myfile:
    slice_public_key=myfile.read().strip()
    

## Setup the Experiment

#### Import FABRIC API

In [3]:
from fabrictestbed_extensions.fablib.fablib import fablib

## Configure Slice Parameters



In [17]:
slice_name = 'MySlice3'
site = 'MAX'
node_name = 'Node1'
image = 'default_ubuntu_20'
username = 'ubuntu'
cores = 2
ram = 8
disk = 10

## Setup the Slice

In [18]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=node_name, site=site)
    node.set_capacities(cores=cores, ram=ram, disk=disk)
    node.set_image(image, username)
    
    #Add an NVME Drive
    node.add_component(model='NVME_P4510', name='nvme1')

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

Waiting for slice ........ Slice state: StableOK
Running post boot config ...Done!


In [9]:
return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    for slice in slices:
        print("{}:".format(slice.slice_name))
        print("   ID         : {}".format(slice.slice_id))
        print("   State      : {}".format(slice.slice_state))
        print("   Lease End  : {}".format(slice.lease_end))
        print()
else:
    print(f"Failure: {slices}")

NameError: name 'slice_manager' is not defined

## Get the New Slice

You new slice is in the list of all your slices. You can loop through the list of slices to get the new slice. Python has a standard tool to filter lists. Try using a lambda function to filter out your slice using its name.

In [19]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Slice: {slice.get_name()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")
    traceback.print_exc()

Slice: MySlice3


## Get the Slice Topology

In [20]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}")  

        print(f"   Interfaces        : {node.get_interfaces()}")
        
        print()    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : Node1
   Cores             : 2
   RAM               : 8
   Disk              : 10
   Image             : default_ubuntu_20
   Image Type        : qcow2
   Host              : max-w4.fabric-testbed.net
   Site              : MAX
   Management IP     : 63.239.135.106
   Reservation ID    : 4d2b05e0-539b-4f11-945b-5d6e82d97fef
   Reservation State : Active
   SSH Command       : ssh -i /home/mono/Sachen/Work/FABRIC/mina_bastion_key -J minawm_0041350787@bastion-1.fabric-testbed.net ubuntu@63.239.135.106
   Components        :  
      Name             : Node1-nvme1
      Details          : Dell Express Flash NVMe P4510 1TB SFF
      Disk (G)         : 960
      Units            : 1
      PCI Address      : 0000:24:00.0
      Model            : P4510
      Type             : NVME
   Interfaces        : []



## Delete Slice

In [22]:
try:
    slice = fablib.get_slice(name='MySlice3')
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()